# 4.1 データセットと前処理 — ハードコーディング版

このノートブックでは、第4.1節の内容に沿って、モデル非依存の前処理パイプラインをハードコーディングで実装します。
- 文字列正規化（Unicode NFKC、空白・改行整形）
- 簡易クリーニング（制御・ゼロ幅文字の除去）
- 厳密重複除去（SHA1）
- 長さ・低情報フィルタ
- シャッフルと train/val 分割
- TXT/JSONL/マニフェストの保存

入出力パスや閾値は下のセルで固定値として定義しています。

In [ ]:
from __future__ import annotations

import json
import hashlib
import random
import re
import unicodedata
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import List

# ハードコーディングされた設定
INPUT_DIR = Path("data/raw")  # 入力となるテキストファイルディレクトリ
OUTPUT_DIR = Path("data/processed")  # 出力先
GLOB = "*.txt"  # 収集する拡張子
TRAIN_RATIO = 0.98
MIN_CHARS = 50
MAX_CHARS = 0  # 0なら無効
UNIQUE_CHAR_RATIO_MIN = 0.01  # 0なら無効
SEED = 42

RE_MULTISPACE = re.compile(r"\s+")
RE_MULTI_NL = re.compile(r"\n{2,}")

def normalize_text(text: str) -> str:
    """基本的な正規化：
    - 改行をLFへ統一
    - Unicode NFKC
    - 行内の空白連続を1つに、前後空白の削除
    - 連続空行の圧縮
    """
    if not text:
        return ""
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = unicodedata.normalize("NFKC", text)
    lines = []
    for line in text.split("\n"):
        line = line.strip()
        if line:
            line = RE_MULTISPACE.sub(" ", line)
        lines.append(line)
    text = "\n".join(lines)
    text = RE_MULTI_NL.sub("\n\n", text)
    return text.strip()

def simple_clean_hooks(text: str) -> str:
    """簡易クリーニング：ゼロ幅文字や制御文字の除去
    （必要に応じてルールを追加）
    """
    text = re.sub(r"[\u200B-\u200D\uFEFF]", "", text)
    text = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", "", text)
    return text

def exact_deduplicate(docs: List[str]) -> List[str]:
    """SHA1による文書単位の厳密重複除去"""
    seen = set()
    uniq = []
    for d in docs:
        key = hashlib.sha1(d.encode("utf-8")).hexdigest()
        if key in seen:
            continue
        seen.add(key)
        uniq.append(d)
    return uniq

def length_filter(docs: List[str], min_chars: int | None, max_chars: int | None, unique_char_ratio_min: float | None):
    out = []
    for d in docs:
        n = len(d)
        if min_chars is not None and n < min_chars:
            continue
        if max_chars is not None and n > max_chars:
            continue
        if unique_char_ratio_min is not None and n > 0:
            ratio = len(set(d)) / n
            if ratio < unique_char_ratio_min:
                continue
        out.append(d)
    return out

def shuffle_and_split(docs: List[str], train_ratio: float, seed: int):
    idx = list(range(len(docs)))
    rng = random.Random(seed)
    rng.shuffle(idx)
    cut = int(len(idx) * train_ratio)
    train = [docs[i] for i in idx[:cut]]
    val = [docs[i] for i in idx[cut:]]
    return train, val

def save_text(path: Path, docs: List[str]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        for d in docs:
            f.write(d.strip())
            f.write("\n\n")  # 空行区切り

def save_jsonl(path: Path, docs: List[str]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        for d in docs:
            f.write(json.dumps({"text": d}, ensure_ascii=False))
            f.write("\n")

def write_manifest(path: Path, manifest: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(manifest, f, ensure_ascii=False, indent=2)


## 入力収集
`INPUT_DIR` 以下から `GLOB` に一致するテキストを再帰的に収集します。各ファイルを1文書として扱います。

In [ ]:
files = sorted(p for p in INPUT_DIR.rglob(GLOB) if p.is_file())
if not files:
    raise FileNotFoundError(f"No input files under {INPUT_DIR} matching {GLOB}")
len(files), files[:5]

## 正規化・クリーニング
UTF-8として読み込み、NFKC正規化→簡易クレンジング→空白・改行整形を適用します。

In [ ]:
raw_docs = []
for p in files:
    text = p.read_text(encoding="utf-8", errors="ignore")
    text = normalize_text(simple_clean_hooks(text))
    if text:
        raw_docs.append(text)
len(raw_docs), sum(len(d) for d in raw_docs)

## 重複除去（厳密）
SHA1キーで厳密な重複を除去します。

In [ ]:
dedup_docs = exact_deduplicate(raw_docs)
len(dedup_docs)

## フィルタリング
- `MIN_CHARS` 未満を除外
- `MAX_CHARS` が正なら上限適用
- `UNIQUE_CHAR_RATIO_MIN` が正ならユニーク文字比の閾値を適用

In [ ]:
max_chars = MAX_CHARS if MAX_CHARS and MAX_CHARS > 0 else None
ucrm = UNIQUE_CHAR_RATIO_MIN if UNIQUE_CHAR_RATIO_MIN and UNIQUE_CHAR_RATIO_MIN > 0 else None
filtered_docs = length_filter(dedup_docs, MIN_CHARS, max_chars, ucrm)
len(filtered_docs)

## 分割（train/val）
シャッフルの上、`TRAIN_RATIO` で分割します。

In [ ]:
train_docs, val_docs = shuffle_and_split(filtered_docs, TRAIN_RATIO, SEED)
len(train_docs), len(val_docs)

## 保存（TXT/JSONL/manifest）
各文書はTXTでは空行区切り、JSONLでは1行1文書(`{text: ...}`)で保存します。

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
save_text(OUTPUT_DIR / "train.txt", train_docs)
save_text(OUTPUT_DIR / "val.txt", val_docs)
save_jsonl(OUTPUT_DIR / "train.jsonl", train_docs)
save_jsonl(OUTPUT_DIR / "val.jsonl", val_docs)
manifest = {
    'total_docs': len(raw_docs),
    'total_chars': int(sum(len(d) for d in raw_docs)),
    'after_dedup_docs': len(dedup_docs),
    'after_filter_docs': len(filtered_docs),
    'train_docs': len(train_docs),
    'val_docs': len(val_docs),
    'params': {
        'train_ratio': TRAIN_RATIO,
        'min_chars': MIN_CHARS,
        'max_chars': MAX_CHARS,
        'unique_char_ratio_min': UNIQUE_CHAR_RATIO_MIN,
        'seed': SEED,
        'glob': GLOB,
    },
}
write_manifest(OUTPUT_DIR / "manifest.json", manifest)
manifest

## プレビュー
前処理済みの先頭の文書を確認します。

In [ ]:
print(train_docs[0][:1000] if train_docs else '(no docs)')
